# How to run LTspice in Python??

To run LTspice in Python you have to install the ***ltspice*** library via the command:

```
pip install ltspice
```

The documantation of the above library is available @ https://pypi.org/project/ltspice/



# DC - Analysis

DC analysis of the following circuit carried out via LTspice and MNA

<img src="DC-simulation_ex1.png" width ="400" height=400>


### Goal 
Compute the DC voltage $V$ and the current $I$


## LTspice netlist and Python
**(file "DC-analysis_ex1.cir")**


```
* DC analysis - SPICE

V1 1 0 10M
R1 1 2 1.5K
R2 2 0 500
I1 2 0 5U

* circuit analysis
.OP
* or
*.DC V1 10M 10M 1

*Saving the data
.PRINT V(1) V(2) I(R2)

.END
```


In [ ]:
import os
import ltspice
import matplotlib.pyplot as plt
import numpy as np


import warnings
warnings.filterwarnings("ignore")

filename = 'DC-analysis_ex1.cir'

#Run the simulation and save in binary .raw
os.system('/Applications/LTspice.app/Contents/MacOS/LTspice -b DC-analysis_ex1.cir')

In [ ]:
#read the raw file generated by LTspice

filepath = 'DC-analysis_ex1.raw'


l = ltspice.Ltspice(filepath)
l.parse() # Data loading sequence. It may take few minutes for huge file.

print(l.variables)

In [ ]:
var = l.variables

e1 = l.getData(var[0])
e2 = l.getData(var[1])

V = e1-e2
I = l.getData(var[2])

print('V:',V)
print('I:',I)

## Modified Nodal Analysis

Structure of the MNA equation:

$\begin{pmatrix}
\mathbf{\bar{G}} &  \mathbf{A}\\
\mathbf{A}^T & \mathbf{0}
\end{pmatrix}
\begin{pmatrix}
\mathbf{e}_n\\
\mathbf{i}
\end{pmatrix}
=
\begin{pmatrix}
\mathbf{a}_S\\
\mathbf{v}_S
\end{pmatrix}
$,

in our case:

$\mathbf{\bar{G}}=\begin{pmatrix}
1/(1.5k) & -1/(1.5k)\\
-1/(1.5k) & 1/(1.5k)+1/500
\end{pmatrix}$,

$\mathbf{A}=\begin{pmatrix}
1\\
0
\end{pmatrix}
$,

$\mathbf{a}_S=\begin{pmatrix}
0\\
5\mu A
\end{pmatrix}$

and $e_S=10mV$.



In [ ]:
G = np.array([[1/1.5e3, -1/1.5e3],[-1/1.5e3, 1/1.5e3+1/500]])

A = np.array([[1],[0]])
                       
MNA_matrix_1 = np.hstack((G,A))
MNA_matrix_2 = np.transpose(np.append(A,0))
    
MNA_matrix = np.vstack((MNA_matrix_1,MNA_matrix_2))       

MNA_matrix

In [ ]:
source = np.array([0, -5e-6,10e-3])
source

In [ ]:
x = np.linalg.solve(MNA_matrix, source.T)

In [ ]:
x

In [ ]:
V = x[0]-x[1]
I = x[1]/500 

print('V:',V)
print('I:', I)